# Laboratório 5

* Daniel Guimarães

* Iury Saboia

## Parte 1: Algoritmo de Deutsch

Combinações de ```f(x)``` possíveis:

```text
f(x) = 0     (constante , variacao 0)
f(x) = 1     (constante , variacao 1)
f(x) = x     (balanceado, variacao 0)
f(x) = 1-x   (balanceado, variacao 1)
```


**Determine o circuito quântico do oráculo Uf correspondente a cada uma das 4 funções f possíveis**

(Para todas as respostas abaixo, foi considerado que um oráculo o Oráculo possui duas entradas, ```[x, y]``` ,
 e duas saídas, ```[x, y ⊕ f(x)]``` )

In [12]:
# Para as opções constantes, basta colocar ou não uma porta inversora na saída:
# Para as opções balanceadas, basta colocar uma porta lógica CNOT

from qiskit import QuantumCircuit
from typing import Union

def aplicando_oraculo(c: Union[QuantumCircuit, None], modo: str="balanceado", variacao: int=0):
    """
    Aplica o oraculo no circuito quântico.
    O circuito quântico precisa de dois qubits, sendo o q0 a entrada x, e o q1 a saída y ⊕ f(x)

    :param c: Circuito quântico. Se for None, um novo circuito quântico sera criado e retornado
    :param modo: "balanceado" ou "constante"
    :param variacao: 0 ou 1
    """

    if c is None:
        c = QuantumCircuit(2)

    if modo == 'constante':
        pass            # joga y na saida
    else:
        c.cnot(0, 1)    # muda o y de acordo com x

    if variacao == 1:
        c.x(1)          # inverte o resultado acima
    c.barrier()

    return c

# qc = QuantumCircuit(2, 2)
# aplicando_oraculo(qc, "balanceado", 0)
# aplicando_oraculo(qc, "balanceado", 1)
# aplicando_oraculo(qc, "constante", 0)
# aplicando_oraculo(qc, "constante", 1)
# qc.draw()

**Simule no Qiskit um circuito quântico que implemente o Algoritmo de Deutsch para cada uma das funções acima.**

In [18]:
from qiskit import Aer, execute

def simula_deutsch(modo: str='balanceado', variacao: int=0):
    """Roda o algoritmo de Deutsch, para uma funcao f(x) determinada pelos argumentos"""

    print(f'Rodando algoritmo de Deutsch para uma função {modo}, com variacao {variacao}:')
    c = QuantumCircuit(2, 1)

    # usando implementação de:
    # https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm#Deutsch's_algorithm
    # e também do qiskit:
    # https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html

    # preparando o circuito com |0⟩|1⟩
    c.x(1)
    c.barrier()
    # aplicando H para ambos os qubits
    c.h(0)
    c.h(1)
    c.barrier()
    # aplicando o oraculo
    c += aplicando_oraculo(None, modo, variacao)
    # aplicando o H no "registrador"
    c.h(0)
    c.barrier()
    # medindo
    c.measure(0, 0)

    # vendo o resultado
    simulator = Aer.get_backend('qasm_simulator')
    resultado = execute(c, simulator).result()
    # resultado deve ser um dicionario, cuja chave única é o resultado.
    try:
        d = dict(resultado.get_counts())
        if len(d.keys()) > 1:
            print("imprecisão no resultado, algoritmo falhou")
        else:
            res = list(d.keys())[0]
            if res == '1':
                print("resultado: funcao balanceada")
            elif res == '0':
                print("resultado: funcao constante")
            else:
                print("resultado invalido: ", res)
    except Exception as e:
        print("erro encontrado: ", e)


simula_deutsch('balanceado', 0)
simula_deutsch('balanceado', 1)
simula_deutsch('constante', 0)
simula_deutsch('constante', 1)

Rodando algoritmo de Deutsch para uma função balanceado, com variacao 0:
resultado: funcao balanceada
Rodando algoritmo de Deutsch para uma função balanceado, com variacao 1:
resultado: funcao balanceada
Rodando algoritmo de Deutsch para uma função constante, com variacao 0:
resultado: funcao constante
Rodando algoritmo de Deutsch para uma função constante, com variacao 1:
resultado: funcao constante


## Parte 2: Algoritmo de Deutsch-Josza

**Crie circuitos quânticos para dois oráculos à sua escolha: um correspondente a uma função constante
e outro correspondente a uma função balanceada, onde as funções têm como entrada sequências de 6 bits,
isto é, f: {0, 1}6 --> {0, 1}.**

Oráculo da função constante:  ```f(x1, x2, ..., x6) = 0```
Oráculo da função balanceada: ```f(x1, x2, ..., x6) = x1    (depende só de x1)```

In [19]:
def aplicando_oraculo_2(c: Union[QuantumCircuit, None], n: int=6, tipo: int=0):
    """
    Aplica o oraculo no circuito quântico.
    Se o circuito for None, será criado um novo circuito.

    :param c: QuantumCircuit, ou None
    :param n: quantidade de bits da sequencia
    :param tipo: 0 para oraculo de funcao constante, 1 para balanceado
    """

    if c is None:
        c = QuantumCircuit(n + 1)       # +1 para o y

    if tipo == 0:   # saida constante
        pass
    else:           # saida balanceada
        c.cnot(0, n)
    c.barrier()
    return c

# num = 6
# qc = QuantumCircuit(num + 1, num)
# qc += aplicando_oraculo_2(None, num, 0)
# qc += aplicando_oraculo_2(None, num, 1)
# qc.draw()

░       ░ 
q_0: ─░───■───░─
      ░   │   ░ 
q_1: ─░───┼───░─
      ░   │   ░ 
q_2: ─░───┼───░─
      ░   │   ░ 
q_3: ─░───┼───░─
      ░   │   ░ 
q_4: ─░───┼───░─
      ░   │   ░ 
q_5: ─░───┼───░─
      ░ ┌─┴─┐ ░ 
q_6: ─░─┤ X ├─░─
      ░ └───┘ ░ 
c: 6/═══════════

**Simule no Qiskit um circuito quântico que implemente o Algoritmo de Deutsch-Josza para
cada uma das funções acima**.

In [21]:
def simula_deutsch_josza(n: int=6, tipo: int=0):
    print(f"Rodando o algoritmo de Deutsch-Jozsa para uma função do tipo {'constante' if tipo == 0 else 'balanceada'}")

    c = QuantumCircuit(n+1, n)

    # preparando o circuito com (|0⟩)^n |1⟩
    c.x(n)
    c.barrier()
    # aplicando H para todos os qubits
    for i in range(n+1):
        c.h(i)
    c.barrier()
    # aplicando o oraculo 2
    c += aplicando_oraculo_2(None, n, tipo)
    # aplicando o H nos qubits de entrada
    for i in range(n):
        c.h(i)
    c.barrier()
    # medindo
    for i in range(n):
        c.measure(i, i)

    # vendo o resultado
    simulator = Aer.get_backend('qasm_simulator')
    resultado = execute(c, simulator).result()
    # resultado deve ser um dicionario, cuja chave unica é o resultado.
    try:
        d = dict(resultado.get_counts())
        if len(d.keys()) > 1:
            print("imprecisao no resultado, algoritmo falhou")
        else:
            res = list(d.keys())[0]
            if res == '0' * n:      # chance de ser 00000..0
                print("resultado: funcao constante")
            else:
                print("resultado: funcao balanceada")
                
    except Exception as e:
        print('erro encontrado: ', e)

simula_deutsch_josza(6, 0)
simula_deutsch_josza(6, 1)

Rodando o algoritmo de Deutsch-Jozsa para uma função do tipo constante
resultado: funcao constante
Rodando o algoritmo de Deutsch-Jozsa para uma função do tipo balanceada
resultado: funcao balanceada
